In [1]:
from __future__ import print_function

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn

In [2]:
sentence=("if you want to build a ship, don't drum up people together to \
collect wood and don't assign them tasks and work, \
but rather teach them to long for the endless immensity of the sea.")

char_set=list(set(sentence))
char_dic={w:i for i,w in enumerate(char_set)}
print(char_set)
print(char_dic)

['m', ' ', '.', 'a', 'y', 't', 'p', 'h', 'd', 'n', 'l', 'g', "'", ',', 'u', 'c', 'i', 'b', 'o', 'k', 'e', 'r', 's', 'w', 'f']
{'m': 0, ' ': 1, '.': 2, 'a': 3, 'y': 4, 't': 5, 'p': 6, 'h': 7, 'd': 8, 'n': 9, 'l': 10, 'g': 11, "'": 12, ',': 13, 'u': 14, 'c': 15, 'i': 16, 'b': 17, 'o': 18, 'k': 19, 'e': 20, 'r': 21, 's': 22, 'w': 23, 'f': 24}


In [4]:
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

dataX = []
dataY = []
for i in range(len(sentence) - sequence_length):
    x_str=sentence[i:i+sequence_length]
    y_str=sentence[i+1:i+sequence_length+1]
    print(i,x_str,'->',y_str)
    
    x=[char_dic[c] for c in x_str]
    y=[char_dic[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)
    #print(x,y)

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [9]:
batch_size=len(dataX)

tf.reset_default_graph()

X=tf.placeholder(tf.int32, [ None,sequence_length])
Y=tf.placeholder(tf.int32, [ None,sequence_length])

X_one_hot=tf.one_hot(X, num_classes)
print(X_one_hot)

def lstm_cell():
    cell=rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    return cell

multi_cells=rnn.MultiRNNCell([lstm_cell() for _ in range(2)], state_is_tuple=True)

outputs, _states=tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)

X_for_fc = tf.reshape(outputs,[-1,hidden_size])
outputs=tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

outputs=tf.reshape(outputs,[batch_size, sequence_length, num_classes])
weights=tf.ones([batch_size,sequence_length])

sequence_loss=tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
mean_loss=tf.reduce_mean(sequence_loss)
train_op=tf.train.AdamOptimizer(learning_rate).minimize(mean_loss)

Tensor("one_hot:0", shape=(?, 10, 25), dtype=float32)


In [10]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _,l,results=sess.run([train_op,mean_loss,outputs], feed_dict={X:dataX,Y:dataY})
    for j,result in enumerate(results):
        index=np.argmax(result, axis=1)
        print(i,j,''.join([char_set[t] for t in index]),1)
    
results=sess.run(outputs, feed_dict={X:dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

0 0 onnngggggg 1
0 1 naaaaagggg 1
0 2 ttcccggggc 1
0 3 olgggggggc 1
0 4 iaaggggkcc 1
0 5 etgggccccc 1
0 6 tggggccccc 1
0 7 gggdkkkccc 1
0 8 mddkkkcccc 1
0 9 akkkkccccc 1
0 10 fccccccccc 1
0 11 tccccccccc 1
0 12 fkkccccccu 1
0 13 iaaatcccum 1
0 14 ttttccccmm 1
0 15 .eeooummmm 1
0 16 eeooummmm, 1
0 17 oooummmmml 1
0 18 uuuuuummll 1
0 19 uumttmmmmm 1
0 20 ttttttmmmm 1
0 21 mtm,,mmmmm 1
0 22 tc'bffmmmm 1
0 23 ffffffmmua 1
0 24 woedddmaaa 1
0 25 ommmmmmaaa 1
0 26 fddmaaaaaa 1
0 27 daaaaaaaaa 1
0 28 ttaaaaaaaa 1
0 29 uukaaaaaaa 1
0 30 iaaaaaaaaa 1
0 31 aaaaaa.aaa 1
0 32 .ftuuuutta 1
0 33 fccuuuttaa 1
0 34 ttttbtaaaa 1
0 35 uuubbaaaaa 1
0 36 .eetteaaaa 1
0 37 eaaaaaaaaa 1
0 38 bbtaaaaaau 1
0 39 tttttaaauu 1
0 40 eeaaaauuuu 1
0 41 fmmuuuuuuu 1
0 42 ttt.uuuuuu 1
0 43 ff.uuuuuuu 1
0 44 ...uuuuuu. 1
0 45 ifuuuuuu.. 1
0 46 fduuuuu... 1
0 47 uuuuus.k.. 1
0 48 ..kkk.kk.. 1
0 49 tccckkk... 1
0 50 fkkkkkk..h 1
0 51 ii.k....hk 1
0 52 p.kk..hhkk 1
0 53 .kf...huuh 1
0 54 ffffhhhhhh 1
0 55 w..hhhhhtt 1
0 